In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from running_embedder import *
import numpy as np

In [ ]:
class BBoxSimilaritySearch(nn.Module):
    def __init__(self, learner):
        super().__init__()
        self.learner = learner
        self.tfms = transforms.Compose([
            partial(crop_pad, size=512, padding_mode="zeros")
        ])
        self.toimg = transforms.ToPILImage()
        self.totsr = transforms.ToTensor()
        
    def forward(self, exemplar:Image, search_target:Image, n=5):
#         exe_embed = self.embedder(exemplar)
#         target_embed = self.embedder(search_target)
#         ans = self.searcher(exe_embed, target_embed)
        # hmm, maybe one backbone and multiple heads?
        # would Centernet itself work well here?
        # concat CenterNet features with Siamese?
        # recurrent bbox controller?
        #  adaptive computation time?
        # higher order training?
        w, h = exemplar.size[1], exemplar.size[0]
        windows = [(x, y, x+w, y+h) for x, y in zip(range(0, search_target.size[1], w), range(0, search_target.size[0], h))]
        search_target_pil = self.toimg(search_target.data)
        crops = list(map(lambda bbox: Image(self.totsr(search_target_pil.crop(bbox))), windows))
        idxs = find_nneighbours(self.learner, exemplar, crops, n=n)
        
        return [crops[i] for i in idxs]

In [ ]:
embedlist = _embed_imgset(learner, imgset)

In [ ]:
bbox_search = BBoxSimilaritySearch(learner)

In [ ]:
img = tfms(PImage.open("images/000000008021-head.jpg"))
target = tfms(PImage.open("images/000000004134.jpg"))
with torch.no_grad():
    candidates = bbox_search(img, target)
display(*candidates)